# Integrated OpenAI API Demo

In [1]:
# pip install pydantic

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from openai import OpenAI
client = OpenAI()

CHAT_MODEL = "gpt-5-nano"

## Structured Output:
**주제/범위/산출물 기대치** 등을 구조화

In [4]:
from pydantic import BaseModel
from typing import List

class ProjectBrief(BaseModel):
    topic: str                # 프로젝트 주제
    goals: List[str]          # 프로젝트의 주요 목표
    deliverables: List[str]   # 산출물(예: 보고서, 발표자료 등)
    constraints: List[str]    # 제약 조건(예: 예산, 일정, 데이터 제한 등)

user_topic = "한국 반도체 산업에서 AI가 미치는 영향 (200자 이내 개요)"

brief_resp = client.responses.parse(
    model=CHAT_MODEL,
    input=[
        {"role": "developer", "content": "다음 입력을 프로젝트 ProjectBrief 로 구조화하세요."},
        {"role": "user", "content": user_topic},
    ],
    text_format=ProjectBrief,  # 모델 출력(JSON)을 ProjectBrief 구조로 파싱하도록 지정
)

# -------------------------------------------------------------
# 모델의 출력 결과 확인
# -------------------------------------------------------------
# - output_text : 모델이 반환한 원본 JSON 문자열
# - output_parsed : JSON을 파싱한 Pydantic 객체 (즉시 Python 객체로 활용 가능)

project_brief = brief_resp.output_parsed

# 모델이 생성한 JSON 문자열 출력 (형태 확인용)
print("[Project Brief]", brief_resp.output_text)

# Pydantic 객체로 파싱된 결과 출력 (Python dict처럼 다룰 수 있음)
print(project_brief)

[Project Brief] {"topic":"한국 반도체 산업에서 AI가 미치는 영향","goals":["한국 반도체 산업의 AI 도입 현황과 영향 요인 식별","생산성, 품질, 비용, 공급망 등 주요 영향 영역의 개요 파악","정책·산업 전략에 대한 시사점 제시","200자 이내 개요 초안 및 최종본 작성"],"deliverables":["200자 이내의 한국어 개요문(한국 반도체 산업에서 AI가 미치는 영향)","요약문의 초안과 최종본","핵심 포인트 목록 및 간단한 해설"],"constraints":["요약문은 200자 이내로 제시","한국어로 작성","객관적이고 사실에 기반한 표현 사용","용어는 간결하고 전문 용어의 남용 방지"]}
topic='한국 반도체 산업에서 AI가 미치는 영향' goals=['한국 반도체 산업의 AI 도입 현황과 영향 요인 식별', '생산성, 품질, 비용, 공급망 등 주요 영향 영역의 개요 파악', '정책·산업 전략에 대한 시사점 제시', '200자 이내 개요 초안 및 최종본 작성'] deliverables=['200자 이내의 한국어 개요문(한국 반도체 산업에서 AI가 미치는 영향)', '요약문의 초안과 최종본', '핵심 포인트 목록 및 간단한 해설'] constraints=['요약문은 200자 이내로 제시', '한국어로 작성', '객관적이고 사실에 기반한 표현 사용', '용어는 간결하고 전문 용어의 남용 방지']


## Conversation State: 이전 응답 ID를 이용해 문맥 잇기
구조화된 `ProjectBrief`를 이어받아, 세부 리서치 체크리스트를 만듭니다.

In [5]:
checklist_resp = client.responses.create(
    model=CHAT_MODEL,  
    input=[
        {"role": "system", "content": "전문 연구 조교로서 간결하고 실행가능한 체크리스트를 제시하세요."},
        # 이전 단계의 JSON 문자열(프로젝트 브리프)을 삽입
        {"role": "user", "content": f"프로젝트 요약: {brief_resp.output_text}"},
    ],
)

# 응답 ID 저장 (대화 문맥 유지용)
previous_response_id = checklist_resp.id

# 생성된 체크리스트 출력
print(checklist_resp.output_text)

다음은 전문 연구 조교로서 제시하는 실행 가능한 체크리스트와 산출물 초안입니다.

1) 실행 체크리스트 (간결하고 실행 가능한 항목)
- 목표 확정 및 범위 설정
  - 한국 반도체 산업에서 AI의 현재 도입 현황과 영향 요인을 파악하되, 생산, 품질, 비용, 공급망 등 핵심 영역에 한정.
- 자료 수집 계획 수립
  - 기업(삼성, SK하이닉스 등), 협회, 정부 정책 문서, 학술/시장 보고서, 사례 연구를 포함한 다源 데이터 확보.
- 영향 영역 정의 및 계량 지표 선정
  - 생산성(OEE), 품질(Yield/Defect Rate), 비용(공정당 비용/비용 절감), 공급망 탄력성(리드타임/재고 회전) 등 측정 지표 도출.
- 현황 분석 프레임워크 구축
  - 도입 수준(적용 공정/설비), 데이터 인프라 상태, 인력 역량, 거버넌스 체계, 보안 이슈 등을 점검.
- 정책·산업 전략 맥락 정리
  - 데이터 거버넌스, 공공-민간 협력, 인재 양성, AI 인프라 투자, 표준화 이니셔티브 등 시사점 도출.
- 개요문(200자) 작성
  - 200자 이내 제시 형식으로 핵심 메시지 요약.
- 요약문 초안 작성
  - 현황, 영향 영역, 주요 요인, 정책 시사점 중심으로 초안 구성.
- 요약문 최종본 다듬기
  - 용어 정리, 중복 제거, 흐름 다듬기, 사실 근거 보강.
- 품질 점검 및 피드백 반영
  - 사실관계 확인, 인용 출처 표기, 이해관계자 리뷰 반영.
- 산출물 정리 및 제출 포맷 확인
  - 200자 개요문, 요약문 초안/최종본, 핵심 포인트 목록 및 해설을 명확한 포맷으로 정리.

2) 200자 이내 개요문 (한국 반도체 산업에서 AI가 미치는 영향)
한국 반도체 산업에서 AI 도입은 생산성 향상, 품질 안정화, 비용 절감, 공급망 관리 개선에 기여하며 설계·생산 공정의 데이터 기반 의사결정과 자동화를 촉진한다. 정책과 인력·데이터 인프라 여건이 도입 속도를 좌우한다.

참고: 아래는 요청에 맞춰 개요문 200자 이내로 유지한 버전입니다.
- 한국 

## Web Search: 최신 출처 수집 + 간단 요약
웹 검색 도구를 활성화하여, 핵심 출처 3~5개를 찾고 한 문단 요약을 받습니다.
반환 객체에 **sources**를 포함시켜 가볍게 인용 목록을 살펴봅니다.

In [6]:
# 이전 단계(ProjectBrief)의 주제(project_brief.topic)를 문자열 내에 삽입
web_query = f"{project_brief.topic} 관련 최신 동향과 핵심 출처 3~5개를 검색해줘."

# 모델 호출 - Web Search Tool 사용
web_resp = client.responses.create(
    model=CHAT_MODEL,
    tools=[{"type": "web_search"}],
    input=[
        {"role": "user", "content": web_query}
    ],
    include=["web_search_call.action.sources"],  # 검색 출처 URL 정보 포함
    previous_response_id=previous_response_id,   # 이전 대화 문맥 유지
)

# 응답 ID 갱신
previous_response_id = web_resp.id

# 모델의 요약 결과 출력
print(web_resp.output_text)

# 웹 검색 출처(URL) 추출
sources = []
for item in getattr(web_resp, "output", []) or []:
    # 출력 중 web_search_call 타입인 항목만 선택
    if getattr(item, "type", "") == "web_search_call":
        action = getattr(item, "action", None)
        if action and getattr(action, "sources", None):
            # 상위 5개의 URL만 추출
            for s in action.sources[:5]:
                sources.append(getattr(s, "url", ""))

print("\n[Top Sources]")
for u in sources:
    print("-", u)

다음은 2025년 11월 8일 기준으로 확인된 한국 반도체 산업의 AI 관련 최신 동향과 이를 뒷받침하는 핵심 출처 4–5개입니다.

최신 동향 요약
- 삼성전자-엔비디아의 AI 주도 제조 가속화
  - 삼성은 AI Megafactory 설립 계획을 발표했고, 제조 전 공정에서 AI를 활용한 분석/예측/최적화를 통해 생산성·품질 관리의 고도화를 추진 중. NVIDIA와의 협력으로 50,000대 규모의 GPU를 활용하는 대규모 AI 인프라를 구축하는 것이 핵심 축으로 제시됨. ([news.samsungsemiconductor.com](https://news.samsungsemiconductor.com/global/samsung-teams-with-nvidia-to-lead-the-transformation-ofglobal-intelligent-manufacturing-through-new-ai-megafactory/?utm_source=openai))

- 삼성 Foundry의 AI 솔루션과 AI 시대 비전
  - SFF 2024를 통해 AI Era를 체계적으로 추진하는 로드맵이 제시되었고, 2nm SF2Z 및 4nm SF4U 같은 첨단 공정 노드 로드맹, AI 솔루션 플랫폼의 도입 등이 강조됨. AI-EDA 도구의 확산과 제조·설계 간 연계 강화가 주목 포인트로 제시됨. ([semiconductor.samsung.com](https://semiconductor.samsung.com/news-events/news/samsung-showcases-ai-era-vision-and-latest-foundry-technologies-at-sff-2024/?utm_source=openai))

- SK hynix의 AI 메모리 리더십 및 메모리 중심 투자
  - SK hynix는 HBM 등 AI 메모리 솔루션으로 글로벌 AI 워크로드를 주도하는 위치를 강화하고 있으며, 서울 외곽의 SEMICON Korea 2025에서도 이에 대한 비전이 강조됨. AI 메모리(PIM, H

## Streaming: 최종 보고서(핵심 요약) 스트리밍 출력
앞서 축적된 대화 상태를 활용해, 토큰 단위로 결과를 출력하는 예입니다.

In [7]:
stream_prompt = (
    "위 결과를 바탕으로, 핵심만 추린 3문장 요약을 작성하세요. "
    "전문 용어는 최소화하고, 불릿 없이 문장으로만 작성."
)

stream = client.responses.create(
    model=CHAT_MODEL,
    input=[{"role": "user", "content": stream_prompt}],
    stream=True,
    previous_response_id=previous_response_id,
)
print("[Streaming] ", end="", flush=True)
for chunk in stream:
    if getattr(chunk, "type", "") == "response.output_text.delta":
        print(chunk.delta, end="", flush=True)
print("\n[done]")

[Streaming] 한국 반도체 산업에서 AI 도입은 생산성·품질 개선과 비용 절감을 이끌고 있으며 삼성전자는 AI 기반 제조를 확대하기 위해 대규모 인프라를 구축하고 있다. 삼성 Foundry의 설계-생산 연계 강화와 SK hynix의 AI 중심 메모리 전략으로 제조와 설계 전반에 AI 활용이 확산되고 있다. 정책과 데이터 인프라, 표준화, 인재 양성 등 생태계 지원이 도입 속도와 효과를 좌우하며 공공-민간 협력이 핵심이다.
[done]


## File Search / Vector Store (RAG): 로컬 폴더 인덱싱 → 질의

In [10]:
# 1) 벡터 스토어 생성
vs = client.vector_stores.create(name="인공지능연구소_eRecruit_매뉴얼")
vector_store_id = vs.id
print("Vector Store:", vector_store_id)

# 2) 파일을 '바이너리 모드'로 열어 파일 객체(스트림)를 만들어야 함
streams = [open("./data/인공지능연구소_eRecruit_매뉴얼.pdf", "rb")]

try:
    # 4) 업로드 + 인덱싱 완료까지 blocking으로 기다림
    batch = client.vector_stores.file_batches.upload_and_poll(
        vector_store_id=vector_store_id,
        files=streams,  # 파일 객체(핸들) 리스트
    )
    print("Upload Status:", getattr(batch, "status", "?"))

    # 파일 카운트/상태 출력
    if hasattr(batch, "file_counts"):
        print("File Counts:", batch.file_counts)

finally:
    # 5) 열린 파일 핸들 닫기 (리소스 누수 방지)
    for s in streams:
        try:
            s.close()
        except Exception:
            pass

Vector Store: vs_690ee32a162481919145325af78e7da6
Upload Status: completed
File Counts: FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


In [11]:
rag_question = f"인공지능 연구소의 eRecuruit 시스템의 특징을 간략히 요약해 주세요."

rag_resp = client.responses.create(
    model=CHAT_MODEL,
    instructions="제공된 file_search 출처를 바탕으로 간결하고 정확하게 답하세요.",
    input=rag_question,
    tools=[{"type": "file_search", "vector_store_ids": [vector_store_id]}],
    include=["file_search_call.results"],
)

previous_response_id = rag_resp.id
print(rag_resp.output_text)

다음은 인공지능 연구소 eRecruit 시스템의 주요 특징 요약입니다.

- 온라인 프로필 기반 지원 시스템: 지원자는 개인 프로필을 만들고, 하나 이상의 채용 공고에 온라인상으로 지원할 수 있습니다. 모든 지원은 온라인으로만 접수됩니다. 

- 채용 공고별 지원 관리: 광고된 채용 공고에 여러 번 지원할 수 있으며, 프로필 정보는 주기적으로 업데이트할 수 있습니다. 

- 브라우저 호환성 및 접근성: Google Chrome 및 Internet Explorer 6 이상에 최적화되었으며, 호환성 보기를 위해 IE9를 권장합니다. 제출 성공 여부 확인에 적합한 브라우저를 사용해야 합니다. 

- 보안 및 데이터 소유권: 사용자의 개인정보와 지원 데이터는 보안 데이터베이스에 저장되며, 데이터의 소유자는 인공지능 연구소입니다. 

- 필수 정보 표기 및 저장 기능: 별표(*)로 표시된 필수 항목은 반드시 입력해야 하며, 지원 과정 중 언제든 저장 후 나중에 계속할 수 있습니다. 

- 계정 관리 및 비밀번호 정책: 비밀번호는 최소 8자 이상이며 문자와 숫자 조합이 권장되며, 변경/잊어버림에 대한 안내와 헬프데스크 지원이 제공됩니다. 사용자 이름으로는 보통 유효한 이메일 주소를 사용하는 것이 좋습니다. fileciteturn0file1turn0file4

- 일정 및 입력 편의 기능: 날짜 선택은 dd/mm/yyyy 방식으로 직접 입력하거나 달력 아이콘으로 선택할 수 있습니다. 

- 조회 및 맞춤법 검사: 조회 기능(돋보기 아이콘)과 맞춤법 검사 기능을 사용해 입력 내용을 점검할 수 있습니다. 

- 드롭다운 안내 및 검색 보조: 드롭다운 옵션이 부족한 경우 가능한 옵션 중에서 선택하고, 필요 시 검색 기능으로 적합한 값을 찾을 수 있습니다. 

- 도움말 및 지원: 시스템 이용에 필요한 일반적 질문은 헬프데스크를 통해 지원받을 수 있습니다. 
